# Exceptions, Debugging, and Performance of py5

In [1]:
import numpy as np
import py5
py5.__version__

'0.3a2'

In [2]:
%load_ext py5

## Example 1

In [3]:
def settings():
    py5.size(500, 500, py5.P2D)

In [4]:
def setup():
    py5.background(255)
    py5.rect_mode(py5.CENTER)
    py5.stroke(128, 32)

The draw function will throw an error when `frame_count == 100`.

In [5]:
def throw_error(p1, p2):
    py5.rect(p1, p2)

def draw():
    py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
    py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)

    if py5.frame_count == 100:
        throw_error('garbage', 'parameters')

    count = 1000
    for i in range(count):
        py5.point(py5.random(py5.width), py5.random(py5.height))

Run the code and observe what is presented to the user.

In [6]:
py5_options = ['--location=1300,50', '--display=1']
py5.run_sketch(py5_options=py5_options)

CRITICAL:py5.methods:File "<ipython-input-5-e8a5623654c7>", line 9, in draw
    4    def draw():
    5        py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
    6        py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)
    7    
    8        if py5.frame_count == 100:
--> 9            throw_error('garbage', 'parameters')
    10   

File "<ipython-input-5-e8a5623654c7>", line 2, in throw_error
    1    def throw_error(p1, p2):
--> 2        py5.rect(p1, p2)
    ..................................................
     p1 = 'garbage'
     p2 = 'parameters'
    ..................................................

TypeError: The parameter types (str, str) are invalid for method rect.
Your parameters must match one of the following signatures:
 * rect(a: float, b: float, c: float, d: float, /) -> None
 * rect(a: float, b: float, c: float, d: float, r: float, /) -> None
 * rect(a: float, b: float, c: float, d: float, tl: float, tr: float, br: float, bl: float, /) -> None


## Debugging

The regular Jupyter notebook debugger works here.

In [7]:
%debug

> <ipython-input-5-e8a5623654c7>(2)throw_error()
      1 def throw_error(p1, p2):
----> 2     py5.rect(p1, p2)
      3 
      4 def draw():
      5     py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)

ipdb> q


## Example 2

Modify the code to throw a different error.

In [8]:
def throw_error(p1, p2):
    print(0 / 0)

In [9]:
py5.run_sketch(py5_options=py5_options)

CRITICAL:py5.methods:File "<ipython-input-5-e8a5623654c7>", line 9, in draw
    4    def draw():
    5        py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
    6        py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)
    7    
    8        if py5.frame_count == 100:
--> 9            throw_error('garbage', 'parameters')
    10   

File "<ipython-input-8-d21bcc29fb0d>", line 2, in throw_error
    1    def throw_error(p1, p2):
--> 2        print(0 / 0)

ZeroDivisionError: division by zero


## Customizing Error Messages

All error messages can be customized.

Custom error messages can (and should) be built into py5 by default.

One can imagine a professor further augmenting the default Python error messages to provide suitable messages for their class or perhaps include links to class lecture notes to assist debugging.

In [10]:
py5.register_exception_msg('ZeroDivisionError', 'Dividing by Zero? Madness!!!')

In [11]:
py5.run_sketch(py5_options=py5_options)

CRITICAL:py5.methods:File "<ipython-input-5-e8a5623654c7>", line 9, in draw
    4    def draw():
    5        py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
    6        py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)
    7    
    8        if py5.frame_count == 100:
--> 9            throw_error('garbage', 'parameters')
    10   

File "<ipython-input-8-d21bcc29fb0d>", line 2, in throw_error
    1    def throw_error(p1, p2):
--> 2        print(0 / 0)

ZeroDivisionError: Dividing by Zero? Madness!!!


## Example 3

In [12]:
def draw():
    py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
    py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)

    count = 2500
    for i in range(count):
        py5.point(py5.random(py5.width), py5.random(py5.height))

In [13]:
py5.run_sketch(py5_options=py5_options)

## Sluggish Sketch

The sketch is slow. Why?

Let's use the built-in line profiler to investigate.

In [14]:
py5.get_frame_rate()

21.304323196411133

In [15]:
py5.profile_draw()

In [16]:
py5.print_line_profiler_stats()

Timer unit: 1e-06 s

Total time: 2.41819 s
File: <ipython-input-12-148949ef5e8b>
Function: draw at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def draw():
     2        39       1065.0     27.3      0.0      py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
     3        39       2091.0     53.6      0.1      py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)
     4                                           
     5        39         43.0      1.1      0.0      count = 2500
     6     95261      71037.0      0.7      2.9      for i in range(count):
     7     95222    2343952.0     24.6     96.9          py5.point(py5.random(py5.width), py5.random(py5.height))



In [17]:
2500 * 26.9

67250.0

The sketch is slow because of the loop drawing one `point` at a time. Each call to `point` is a call from Python to Java, which has a small overhead. That overhead times 2,500 adds up to a performance penalty.

To counter this, py5 has some built-in functions such as `points` and `vertices` to speed up common use cases.

Users will also be able to build their own Java extensions to speed up critical parts of their code.

In [18]:
def draw():
    py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
    py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)

    count = 2500
    random_coords = py5.width * np.random.rand(count, 2)
    py5.points(random_coords)

In [19]:
py5.run_sketch(py5_options=py5_options)

In [20]:
py5.get_frame_rate()

60.07355880737305

In [21]:
py5.profile_draw()

In [22]:
py5.print_line_profiler_stats()

Timer unit: 1e-06 s

Total time: 0.906752 s
File: <ipython-input-18-6ed870eb14fc>
Function: draw at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def draw():
     2        98       3425.0     34.9      0.4      py5.fill(py5.random(255), py5.random(255), py5.random(255), 50.0)
     3        98       4649.0     47.4      0.5      py5.rect(py5.mouse_x, py5.mouse_y, 40, 40)
     4                                           
     5        98        118.0      1.2      0.0      count = 2500
     6        98       8962.0     91.4      1.0      random_coords = py5.width * np.random.rand(count, 2)
     7        98     889598.0   9077.5     98.1      py5.points(random_coords)



## Discussion

This performance is comparable to the equivalent Processing code, which also gets 60 fps on my computer. 

```
    void setup() {
      size(500, 500, P2D);
      background(255);
      rectMode(CENTER);
      stroke(128, 32);
    }

    void draw() {
      fill(random(255), random(255), random(255), 50.0);
      rect(mouseX, mouseY, 40, 40);

      int count = 2500;
      for (int i = 0; i < count; i++) {
        point(random(width), random(height));
      }
    }

    void keyPressed() {
      println(frameRate);
    }
```

However, if `count` is increased further, the py5 code above will not be able to keep up with Processing's performance.

Consider that much of the time needed for the `points` method is because JPype must use JNI to repeatedly transfer 5000 floats from Python's memory space to the JVM. If thousands of random points are necessary, this can be further improved using custom Java code employing [Java direct buffers, which JPype supports](https://jpype.readthedocs.io/en/latest/userguide.html#buffer-backed-numpy-arrays).